# Decision Tree from the scratch only by Pandas&Numpy


# 0. Import libraries

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import random
from pprint import pprint

# 0. Load data and prepare data

In [2]:
df = pd.read_csv("./data/Titanic.csv")
df["label"] = df.Survived # label ==  if survised
df = df.drop(["PassengerId", "Survived", "Name", "Ticket", "Cabin"], axis=1)
df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,label
0,3,male,22.0,1,0,7.2500,S,0
1,1,female,38.0,1,0,71.2833,C,1
2,3,female,26.0,0,0,7.9250,S,1
3,1,female,35.0,1,0,53.1000,S,1
4,3,male,35.0,0,0,8.0500,S,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    object 
 2   Age       714 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Embarked  889 non-null    object 
 7   label     891 non-null    int64  
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In the infomation table of the raw dataset, we can see the column **Age** and **Embarked** involve missing values. The missing value of age will be filled by the median, and the missing values of embarked will be filled by the highest frequency value in the existing values.

In [4]:
# fill missing values
median_age = df.Age.median()
mode_embarked = df.Embarked.mode()[0]

df = df.fillna({"Age": median_age, "Embarked": mode_embarked})

# 0. Train-test-split

In [5]:
def train_test_split(df, test_size):
    
    if isinstance(test_size, float):# test_size is the ratio of training set and test set
        test_size = round(test_size * len(df))

    indices = df.index.tolist()
    test_indices = random.sample(population=indices, k=test_size)

    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)
    
    return train_df, test_df

In [6]:
train_df, test_df = train_test_split(df, test_size=0.2)

# 1. Helper functions:
## How many potential split options?

In [7]:
def get_potential_splits(df):
    potential_splits = {}
    _, n_columns = df.shape # n_columns - 1 is the number of features 
    for column_index in range(n_columns - 1):    
        values = df.iloc[:,column_index]
        unique_values = np.unique(values)
        
        potential_splits[column_index] = unique_values
    
    return potential_splits

## Calculate impurity using the Entropy
$Entropy_i=sum(-P_ilog_2P_i)$ of each part of one split

$Overal_{Entropy} = sum(P_iEntropy_i) $

In [8]:
def entropy(df):
    y = df.iloc[:,-1]
    p = y.value_counts()/len(y) # the percentage of each distinct value among the whole
    entropy = np.sum(-p*np.log2(p))
    return entropy

In [9]:
def get_impurity(df, feature_index_split, value_split):
    feature_name = df.columns[feature_index_split]
    values = df.iloc[:,feature_index_split]
    if values.dtype == 'O': 
        # if categorical feature
        below_data = df[df[feature_name] == value_split]
        above_data = df[df[feature_name] != value_split]
    else:
        # if cotinuous
        below_data = df[df[feature_name] <= value_split]
        above_data = df[df[feature_name] > value_split]
        
    ##
    n = len(values)
    p_below_data = len(below_data) / n
    p_above_data = len(above_data) / n

    impurity =  p_below_data * entropy(below_data) + p_above_data * entropy(above_data)
    
    return impurity

## Get the best split option

In [10]:
def get_best_split(df, potential_splits):
    
    impurity = 9999
    for feature_index in potential_splits:
        for value in potential_splits[feature_index]:
            current_impurity = get_impurity(df, feature_index_split=feature_index, value_split=value)

            if current_impurity <= impurity:
                impurity = current_impurity
                best_split_feature_index = feature_index
                best_split_value = value
    
    return best_split_feature_index, best_split_value

# Split the data based on the best split option

In [11]:
def get_below_above_data(df, best_split_feature_index, best_split_value):
    feature_name = df.columns[best_split_feature_index]
    values = df.iloc[:,best_split_feature_index]
    if values.dtype == 'O': 
        # if categorical feature
        below_data = df[df[feature_name] == best_split_value]
        above_data = df[df[feature_name] != best_split_value]
    else:
        # if cotinuous
        below_data = df[df[feature_name] <= best_split_value]
        above_data = df[df[feature_name] > best_split_value]
    return below_data, above_data

## Get the most probable(frequent) class

In [12]:
def classify_data(df): # the classified label is the most frequent element
    labels = df.iloc[:, -1]
    
    unique_classes, counts_unique_classes = np.unique(labels, return_counts=True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    return classification

## Stop conditions
- The sampled data set is totally pure.
- The size of the sampled data set is small enough. (min_sample)
- The depth of the tree is big enough. (max_depth)

In [13]:
def get_pure(df):
    labels = df.iloc[:, -1]
    unique_labels = set(labels)
    
    if len(unique_labels) == 1:
        return True
    else:
        return False

# 2. Decison Tree Algorithm 

In [14]:
def decision_tree(df, current_depth=0, min_samples=2, max_depth=5):
    # check stop conditions
    if (get_pure(df)) or (len(df) < min_samples) or (current_depth == max_depth):
        classification = classify_data(df)
        return classification
    
    # recursive part
    else:
        current_depth += 1
        
        # split the data into yes answer or no answer
        potential_splits = get_potential_splits(df)
        best_split_feature_index, best_split_value = get_best_split(df, potential_splits)
        below_data, above_data = get_below_above_data(df, best_split_feature_index, best_split_value)
        
        # check for empty data
        if len(below_data) == 0 or len(above_data) == 0:
            classification = classify_data(df)
            return classification


        # determine question
        feature_name = df.columns[best_split_feature_index]
        feature_type = df.iloc[:,best_split_feature_index].dtype
        if feature_type == 'O': # if categorical
            question = "{} = {}".format(feature_name, best_split_value)
        else:
            question = "{} <= {}".format(feature_name, best_split_value)
        
        # instantiate sub-tree
        sub_tree = {question: []}
        
        # find answers (recursion)
        yes_answer = decision_tree(below_data, current_depth, min_samples, max_depth)
        no_answer = decision_tree(above_data, current_depth, min_samples, max_depth)
        
        # If the answers are the same, then there is no point in asking the qestion.
        # This could happen when the data is classified even though it is not pure
        # yet (min_samples or max_depth base case).
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree

In [15]:
tree = decision_tree(train_df, max_depth=10)
pprint(tree)

{'Sex = male': [{'Age <= 6.0': [{'SibSp <= 2': [1,
                                                {'Age <= 2.0': [0,
                                                                {'Fare <= 27.9': [0,
                                                                                  1]}]}]},
                                {'Pclass <= 1': [{'Fare <= 25.925': [0,
                                                                     {'Fare <= 26.3875': [1,
                                                                                          {'Fare <= 263.0': [{'Fare <= 133.65': [{'Age <= 25.0': [{'Fare <= 53.1': [0,
                                                                                                                                                                    1]},
                                                                                                                                                  {'Age <= 60.0': [{'Fare <= 26.55': [1,
                    

In [18]:
def classify_example(example, tree):
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split(" ")

    # ask question
    if comparison_operator == "<=":  # feature is continuous
        if example[feature_name] <= float(value):
            answer = tree[question][0]
        else:
            answer = tree[question][1]
    
    # feature is categorical
    else:
        if str(example[feature_name]) == value:
            answer = tree[question][0]
        else:
            answer = tree[question][1]

    # base case
    if not isinstance(answer, dict):
        return answer
    
    # recursive part
    else:
        residual_tree = answer
        return classify_example(example, residual_tree)

In [19]:
def calculate_accuracy(df, tree):

    df["classification"] = df.apply(classify_example, axis=1, args=(tree,))
    df["classification_correct"] = df["classification"] == df["label"]
    
    accuracy = df["classification_correct"].mean()
    
    return accuracy

In [20]:
accuracy = calculate_accuracy(test_df, tree)
accuracy

0.7808988764044944